markstro
3/27/2020

This notebook is for estimating the parameter "dday_slope" for GF v1.1



Compute the daily radadj value for each HRU for each day

In [1]:
import numpy as np
import pandas as pd
import fiona 
import geopandas as gpd
import math
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [2]:
# File written by dday_slope-radadj.ipynb
radadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_GF_v1.1.csv'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

In [3]:
coolwarm = cm.get_cmap('coolwarm', 12)
print(coolwarm)

In [4]:
# Get all the layers from the .gdb file 
layers = fiona.listlayers(gdb_path)
print(layers)

['TBtoGFv1_POIs', 'nhru_v11', 'nsegment_v11', 'POIs_GFv11']


In [5]:
nhru_v11_df = gpd.read_file(gdb_path,layer='nhru_v11')

In [6]:
# read the rdadj values from the csv file produced by dday_slope-radadj.ipynb
radadj_df = pd.read_csv(radadj_fn)
radadj_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,358,359,360,361,362,363,364,365,366,nhru_v11
0,0,0.555487,0.553955,0.552431,0.550683,0.548830,0.546875,0.544821,0.542668,0.540421,...,0.513241,0.512909,0.512468,0.511917,0.511258,0.510381,0.509508,0.508420,0.507229,76127
1,1,0.516948,0.515722,0.514298,0.512882,0.511274,0.509676,0.507889,0.506116,0.504159,...,0.473225,0.472941,0.472562,0.472089,0.471524,0.470772,0.470022,0.469088,0.468065,76147
2,2,0.567159,0.565695,0.563996,0.562187,0.560270,0.558248,0.556123,0.553897,0.551457,...,0.517121,0.516783,0.516332,0.515769,0.514984,0.514200,0.513197,0.512087,0.510871,76170
3,3,0.583345,0.581800,0.580007,0.578099,0.576077,0.573944,0.571579,0.569234,0.566664,...,0.529518,0.529162,0.528689,0.528097,0.527272,0.526449,0.525395,0.524228,0.522951,76172
4,4,0.524481,0.523234,0.521786,0.520346,0.518813,0.517085,0.515269,0.513465,0.511476,...,0.475030,0.474744,0.474363,0.473888,0.473318,0.472656,0.471807,0.470867,0.469931,76181


In [7]:
nhru_v11_df = nhru_v11_df.join(radadj_df.set_index('nhru_v11'), on='nhru_v11')

In [8]:
nhru_v11_df.head()

,nhru_v11,hru_segment_GFv11,nhm_id,hru_id_nat,Version,Shape_Length,Shape_Area,geometry,Unnamed: 0,1,...,357,358,359,360,361,362,363,364,365,366
0,76127,40038,76128,76128,1,92601.509382,1.881513e+08,"MULTIPOLYGON (((-105544.567 804074.976, -10554...",0,0.555487,...,0.513462,0.513241,0.512909,0.512468,0.511917,0.511258,0.510381,0.509508,0.508420,0.507229
1,76147,40038,76148,76148,1,60460.683381,4.416189e+07,"MULTIPOLYGON (((-97185.217 806355.005, -97184....",1,0.516948,...,0.473415,0.473225,0.472941,0.472562,0.472089,0.471524,0.470772,0.470022,0.469088,0.468065
2,76170,40021,76171,76171,1,62333.253245,7.337575e+07,"MULTIPOLYGON (((-106590.453 815804.909, -10646...",2,0.567159,...,0.517234,0.517121,0.516783,0.516332,0.515769,0.514984,0.514200,0.513197,0.512087,0.510871
3,76172,40019,76173,76173,1,36798.115971,3.627274e+07,"MULTIPOLYGON (((-106762.641 815925.020, -10675...",3,0.583345,...,0.529756,0.529518,0.529162,0.528689,0.528097,0.527272,0.526449,0.525395,0.524228,0.522951
4,76181,40019,76182,76182,1,31441.172667,1.564925e+07,"MULTIPOLYGON (((-109785.311 816675.047, -10978...",4,0.524481,...,0.475221,0.475030,0.474744,0.474363,0.473888,0.473318,0.472656,0.471807,0.470867,0.469931
